In [1]:
import numpy as np
import pandas as pd
from numpy import ndarray, float32

In [2]:
"""Activation and cost functions."""
    
class MSE():
    """Mean Squared Error."""

    def __str__(self) -> str:
        return "mse"
    
    @staticmethod
    def __call__(e):
        """Mean Squared Error.
        
        `1/2 * sum(e^2)`.
        """
        return np.divide(np.sum(e**2), 2)
    
    @staticmethod
    def diff(e):
        """Derivative of Mean Squared Error.
        
        `e`.
        """
        return e

class Sigmoid():
    """Sigmoid."""
    def __str__(self) -> str:
        return "sigmoid"
    
    @staticmethod
    def __call__(x):
        """Sigmoid activation function.

        `1 / (1 + e^-x)`
        
        Args:
            x `<ndarray[float32]`: `wx + b` vectorized for our use case
        
        Returns:
            `sig(x)`
        """

        return np.divide(1, np.add(1, np.exp(-x)))
    
    @staticmethod
    def diff(x):
        """Derivative of the Sigmoid activation function.

        `e^-x / (e^-x + 1)^2`
        
        Args:
            x `<ndarray[float32]`: `wx + b` vectorized for our use case
        
        Returns:
            `sig'(x)`
        """
        return np.divide(np.exp(-x), np.power(np.add(np.exp(-x), 1), 2))

class ReLU():
    """ReLU."""
    def __str__(self) -> str:
        return "relu"
    
    @staticmethod
    def __call__(x):
        return x if x>0 else 0

    @staticmethod
    def diff(x):
        return 1 if x>0 else 0

In [23]:
"""Network classes."""

class Layer():
    "Class representing the weights and biases of a layer of the MLP using numpy"
    def __init__(self, shape:tuple[int], activation) -> None:
        """MLP Layer contructor
        
        Args:
            shape `<tuple[int]>`: shape of the weights/biases
        """
        self.shape = shape
        self.activation = activation
        self.weights:ndarray[float32] = np.random.rand(shape[0], shape[1])
        self.biases:ndarray[float32] = np.random.rand(1, shape[0])

    def __str__(self) -> str:
        return str(self.shape) + str(self.activation)

    def compute(self, x:ndarray[float32]) -> ndarray[float32]:
        """Calculating `wx + b`
        
        Args:
            x `<ndarray[float32]>`: input data
        """

        return np.add(np.matmul(x, np.transpose(self.weights)), self.biases)

class MLP():
    """Class representing an MLP with the preconfiguration from the assignment."""
    def __init__(self, layers:ndarray[tuple[float32, any]]) -> None:
        """Building an MLP with the given layer/neuron config.
        
        Args:
        - layers `<ndarray[tuple[float32]]>`: array representing the neurons of each layer."""
        
        
        self.layers = [Layer((layers[i+1][0], layers[i][0]), layers[i][1]) for i in range(len(layers)-1)]

    def __str__(self) -> str:
        return str([str(l) for l in self.layers])

    def feed_forward(self, x:ndarray[float32]):
        """One cycle of feed forward.
        
        Args:
            x `<ndarray[float32]>: One batch of training data.
            
        Returns:
            y_pred `<float32>`: Predicted y value.
        """
        #TODO
        return 

    def back_propagation():
        pass

layers = np.array([(2, Sigmoid()), (10, Sigmoid()), (10, Sigmoid()), (2, Sigmoid())])
mlp = MLP(layers)